In [333]:
import pandas as pd
import numpy as np
from itertools import permutations, combinations, product

In [231]:
year = 2022

Bring in our predictions

In [232]:
df_pred = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness/data/2022/predictions/phase2_submissions_xg.csv')
df_pred.head()

,ID,Pred
0,2022_1103_1104,0.159308
1,2022_1103_1112,0.064552
2,2022_1103_1116,0.182135
3,2022_1103_1120,0.061031
4,2022_1103_1124,0.092754


Excellent, now we have our submission file into a dataframe with ID and Predictions. From here we'll need to bring in the first round teams and see who we predict will win.

In [349]:
#read in the team data
df_teams = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MTeams.csv')
df_teams.head(20)

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023
5,1106,Alabama St,1985,2023
6,1107,SUNY Albany,2000,2023
7,1108,Alcorn St,1985,2023
8,1109,Alliant Intl,1985,1991
9,1110,American Univ,1985,2023


In [234]:
#read in the tournament slot data
df_slots = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MNCAATourneySlots.csv')
df_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [235]:
df_slots_thisyear = df_slots[df_slots['Season']==year]
df_slots_thisyear = df_slots_thisyear.reset_index(drop=True)
df_slots_thisyear.head(48)

,Season,Slot,StrongSeed,WeakSeed
0,2022,R1W1,W01,W16
1,2022,R1W2,W02,W15
2,2022,R1W3,W03,W14
3,2022,R1W4,W04,W13
4,2022,R1W5,W05,W12
5,2022,R1W6,W06,W11
6,2022,R1W7,W07,W10
7,2022,R1W8,W08,W09
8,2022,R1X1,X01,X16
9,2022,R1X2,X02,X15


In [236]:
#read in the tournament seed data data
df_seeds = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MNCAATourneySeeds.csv')
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [237]:
df_seeds_thisyear = df_seeds[df_seeds['Season']==year]
df_seeds_thisyear = df_seeds_thisyear.reset_index(drop=True)
df_seeds_thisyear.head(20)

,Season,Seed,TeamID
0,2022,W01,1124
1,2022,W02,1246
2,2022,W03,1345
3,2022,W04,1417
4,2022,W05,1388
5,2022,W06,1400
6,2022,W07,1293
7,2022,W08,1314
8,2022,W09,1266
9,2022,W10,1362


Eliminate the letters for the playin games

In [238]:
df_seeds_thisyear.Seed.replace(['W12a','Y16a','X11a','Z16a'],['W12','Y16','X11','Z16'],inplace=True)
vals_to_drop=['W12b','X11b','Y16b','Z16b']
df_seeds_thisyear = df_seeds_thisyear[df_seeds_thisyear.Seed.isin(vals_to_drop) == False]
df_seeds_thisyear.index = range(64)

In [350]:
print(df_seeds_thisyear[31:67])

    Season Seed  TeamID
31    2022  X16    1209
32    2022  Y01    1242
33    2022  Y02    1120
34    2022  Y03    1458
35    2022  Y04    1344
36    2022  Y05    1234
37    2022  Y06    1261
38    2022  Y07    1425
39    2022  Y08    1361
40    2022  Y09    1166
41    2022  Y10    1274
42    2022  Y11    1235
43    2022  Y12    1350
44    2022  Y13    1355
45    2022  Y14    1159
46    2022  Y15    1240
47    2022  Y16    1394
48    2022  Z01    1112
49    2022  Z02    1437
50    2022  Z03    1397
51    2022  Z04    1228
52    2022  Z05    1222
53    2022  Z06    1161
54    2022  Z07    1326
55    2022  Z08    1371
56    2022  Z09    1395
57    2022  Z10    1260
58    2022  Z11    1276
59    2022  Z12    1412
60    2022  Z13    1151
61    2022  Z14    1255
62    2022  Z15    1174
63    2022  Z16    1136


# Helpful Definitions

In [240]:
'''Make a dictionary of the rank of each team and the probability that they win the round'''
def prob_win_rank(rank1, rank2, dict_region, dict_prob):
    teamid1 = dict_region[rank1]
    teamid2 = dict_region[rank2]
    prob_teamid1, prob_teamid2 = prob_win(teamid1,teamid2)
    dict_prob.update({rank1: prob_teamid1,rank2: prob_teamid2})
    return dict_prob

In [241]:
'''Given two teamIDs, calculate the probabality that each team wins the matchup'''
def prob_win(teamid1=int,teamid2=int):
    if teamid1 < teamid2:
        id_pred = str(year)+'_'+str(teamid1)+'_'+str(teamid2)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        prob_teamid1 = pred
        prob_teamid2 = 1-pred
    elif teamid2 < teamid1:
        id_pred = str(year)+'_'+str(teamid2)+'_'+str(teamid1)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        prob_teamid2 = pred
        prob_teamid1 = 1-pred
    return prob_teamid1, prob_teamid2

In [242]:
'make a dictionary of the team seed and their id for each region'
def seed_id(int1=int, int2=int):
    seeds = [i+1 for i in range(16)]
    TeamIDs = [int(df_seeds_thisyear.TeamID[j]) for j in range(int1,int2)]
    dict_out = dict(zip(seeds,TeamIDs))
    return dict_out

In [243]:
'given a tuple, give the list containing the list of matchups in the next round'
def tup_to_matchups(tup=tuple, half_len_of_tuple=int):
    matchup_list = [[tup[i],tup[-i-1]] for i in range(half_len_of_tuple)]
    return matchup_list

In [268]:
'''function to calculate all the possible round results and their probabilities'''
def round_results(round_num=int, half_of_teams_in_round=int, tup_match=tuple, prob=float, round_results_dict=dict):
    #calculate all possible combinations of ranks in tup_match as a list of tuples
    round_matchups = tup_to_matchups(tup_match,half_of_teams_in_round)
    possible_combos = list(product(*round_matchups))
    '''use dict to pull the probability that each team in the tuple wins and multiply together to 
    calculate the probability of that tuple'''
    for i in possible_combos:
        #start with probability 1
        prob_combo = 1
        #calculate the probablity of each matchup and multiply together to get the probablity of the tuple emmerging from the round
        for j in i:
            prob_j = round1_prob_by_rank[j]
            prob_combo = prob_combo*prob_j
        #multiply with the probability of tup_match from the previous round(s) to get the probability for that version of the bracket
        if round_num==1:
            prob_start = 1
        else:
            prob_start = prob
        prob_combo = prob_start*prob_combo
        nest_tup = tup_match, i
        dict_temp = {nest_tup:prob_combo}
        round_results_dict.update(dict_temp)
    return(round_results_dict)

# Round: 1
For each match up in each region, we'll calculate the possible outcome tuples and the probabilities of each tuple occuring. Because the number of possible brackets is enormous, we'll eliminate the tuples that are highly unlikely in order to reduce the computation needs moving forward. 

In [353]:
'Calculate the rank and team id for each region'
W_region = seed_id(0,16)
X_region = seed_id(16,32)
Y_region = seed_id(32,48)
Z_region = seed_id(48,64)
W_region

{1: 1124,
 2: 1246,
 3: 1345,
 4: 1417,
 5: 1388,
 6: 1400,
 7: 1293,
 8: 1314,
 9: 1266,
 10: 1362,
 11: 1439,
 12: 1231,
 13: 1103,
 14: 1463,
 15: 1389,
 16: 1313}

In [269]:
tup_round1 = tuple(i+1 for i in range(16))
round1_W = {}
round1_W = round_results(1, 8, tup_round1, 1, round1_W)
print(round1_W)

{((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 6, 7, 8)): 0.06504348555154586, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 6, 7, 9)): 0.07725458376915963, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 6, 10, 8)): 0.06155236199151859, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 6, 10, 9)): 0.07310804556891393, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 11, 7, 8)): 0.022393146688629403, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 11, 7, 9)): 0.0265971789802197, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 11, 10, 8)): 0.021191223985303993, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 5, 11, 10, 9)): 0.02516961037160751, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), (1, 2, 3, 4, 12, 6, 7, 8)): 0.04627663625110452, ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11

In [143]:
'only keep tuples with a probability of occuring greater than 0.001'
round1_W = {key:val for key, val in round1_results_pred_W.items() if val >= 0.001}
round1_X = {key:val for key, val in round1_results_pred_X.items() if val >= 0.001}
round1_Y = {key:val for key, val in round1_results_pred_Y.items() if val >= 0.001}
round1_Z = {key:val for key, val in round1_results_pred_Z.items() if val >= 0.001}

# Round 2
For each tuple of teams in the round1 dictionaries, calculate the matchups, the probability of each team winning the second round, possible tuples to emmerge from round2, and probabilities of it happening

In [270]:
round2_W = {}
for key, val in round1_W.items():
    tup_seeds = key[1]
    prob_tup = val
    round2_W = round_results(2, 4, tup_seeds, prob_tup, round2_W)
print(round2_W)
print(len(round2_W))

{((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 4)): 0.04144555412877649, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 5)): 0.02731702858728388, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 4)): 0.03451735548649301, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 5)): 0.022750608754131412, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 4)): 0.024406584898570366, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 5)): 0.01608653548992614, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 4)): 0.020326686055098356, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 5)): 0.013397448187725761, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 4)): 0.020539474707120257, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 5)): 0.013537698542981499, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 4)): 0.01710601691483359, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 5)): 0.011274684653151373, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 4)): 0.012095348795525153, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 5)): 0.007972121395552065, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 4)): 0.010073443651182562, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 5)): 0.

In [226]:
round2_W = {key:val for key, val in round2_W.items() if val >= 0.001}
print(round2_W)
print(len(round2_W))

{((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 4)): 0.04144555412877649, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 5)): 0.02731702858728388, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 4)): 0.03451735548649301, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 5)): 0.022750608754131412, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 4)): 0.024406584898570366, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 5)): 0.01608653548992614, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 4)): 0.020326686055098356, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 5)): 0.013397448187725761, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 4)): 0.020539474707120257, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 5)): 0.013537698542981499, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 4)): 0.01710601691483359, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 5)): 0.011274684653151373, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 4)): 0.012095348795525153, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 5)): 0.007972121395552065, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 4)): 0.010073443651182562, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 5)): 0.

# Reverse engineer from the winner

Our prediction file does most of this work for us since we calculate the probability that each team combination wins in that combination. 

In [273]:
df_pred = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness/data/2022/predictions/phase2_submissions_xg.csv')
df_pred.head(70)

,ID,Pred
0,2022_1103_1104,0.159308
1,2022_1103_1112,0.064552
2,2022_1103_1116,0.182135
3,2022_1103_1120,0.061031
4,2022_1103_1124,0.092754
...,...,...
65,2022_1103_1461,0.207871
66,2022_1103_1463,0.392496
67,2022_1104_1112,0.257408
68,2022_1104_1116,0.467675


If I want to know which team is most likely to win overall, I want to pick the best team which in this case is the team that is most likely to be able to beat all the other teams. Therefore if I add the probabilities of each team against all other teams, and take the maximum, that will tell me the overall dominant team.

In [322]:
new = df_pred["ID"].str.split("_", expand = True)

# separate the ID column into the two different teams and the prediction, they'll be strings so make it numeric
df_pred_expand = pd.DataFrame()
df_pred_expand["year"]= new[0]
df_pred_expand['TeamID1'] = new[1]
df_pred_expand['TeamID2'] = new[2]
df_pred_expand['Pred']= df_pred.Pred
df_pred_expand = df_pred_expand.apply(pd.to_numeric)
df_pred_expand

,year,TeamID1,TeamID2,Pred
0,2022,1103,1104,0.159308
1,2022,1103,1112,0.064552
2,2022,1103,1116,0.182135
3,2022,1103,1120,0.061031
4,2022,1103,1124,0.092754
...,...,...,...,...
2273,2022,1458,1461,0.547007
2274,2022,1458,1463,0.750068
2275,2022,1460,1461,0.183302
2276,2022,1460,1463,0.355910


In [336]:
#make an array of all team IDs
teams = df_pred_expand.TeamID1.unique()
teams = np.append(teams,df_pred_expand.TeamID2.max())
teams

array([1103, 1104, 1112, 1116, 1120, 1124, 1129, 1136, 1151, 1159, 1161,
       1163, 1166, 1168, 1172, 1174, 1181, 1209, 1211, 1222, 1228, 1231,
       1234, 1235, 1240, 1242, 1246, 1255, 1260, 1261, 1266, 1272, 1274,
       1276, 1277, 1286, 1293, 1308, 1313, 1314, 1323, 1326, 1344, 1345,
       1350, 1353, 1355, 1361, 1362, 1371, 1388, 1389, 1394, 1395, 1397,
       1400, 1403, 1411, 1412, 1417, 1425, 1436, 1437, 1439, 1458, 1460,
       1461, 1463])

In [337]:
def create_team_df(teamid = int):
    #make a dataframe with just the rows that contain a specific teamid
    mask = ((df_pred_expand.TeamID1 == teamid) | (df_pred_expand.TeamID2 == teamid))
    team_df = df_pred_expand[mask].reset_index(drop = True)
    return team_df

In [341]:
def pred_teamid_wins(team_df, teamid = int):
    for i in range(len(team_df.year)):
        if team_df.TeamID2[i] == teamid:
            team_df.Pred[i] = 1-team_df.Pred[i]
    return team_df

In [344]:
domination_prob = {}
for team in teams:
    team_df = create_team_df(team)
    team_df = pred_teamid_wins(team_df, team)
    domination_prob.update({team:sum(team_df.Pred)})
domination_prob

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{1103: 15.781158830000003,
 1104: 37.37110307999998,
 1112: 49.413500150000004,
 1116: 35.578808370000004,
 1120: 55.059175528000004,
 1124: 52.50637138000001,
 1129: 34.18272025,
 1136: 16.996861465000006,
 1151: 21.879821425999992,
 1159: 18.041995465000007,
 1161: 33.74638674999999,
 1163: 39.61901346,
 1166: 23.08743629000001,
 1168: 18.045282033,
 1172: 33.34723209000002,
 1174: 19.058536282000006,
 1181: 49.39073294800001,
 1209: 19.678046063,
 1211: 54.39782426099999,
 1222: 48.60614321499998,
 1228: 42.713950194999995,
 1231: 34.91201538,
 1234: 36.05487480000001,
 1235: 36.85962672,
 1240: 18.93896246,
 1242: 52.41447743299999,
 1246: 50.24817038199999,
 1255: 20.164002249999996,
 1260: 36.84900547,
 1261: 41.60912394000001,
 1266: 38.597975500000004,
 1272: 30.29856450999999,
 1274: 31.580032979999995,
 1276: 31.22156766000001,
 1277: 40.76967036599999,
 1286: 18.94211826600001,
 1293: 36.799003889999994,
 1308: 24.95636032000001,
 1313: 19.287672729999997,
 1314: 34.24299110

In [345]:
max(domination_prob, key=domination_prob.get)

1120

In [348]:
df_teams.TeamName[df_teams.TeamID == 1120]

19    Auburn
Name: TeamName, dtype: object

Seth's idea is to take the minimum probability for each team and select the team that has the highest one since that team would be the most difficult to elimanate. Let's try that!

In [351]:
minimum_prob = {}
for team in teams:
    team_df = create_team_df(team)
    team_df = pred_teamid_wins(team_df, team)
    minimum_prob.update({team:min(team_df.Pred)})
minimum_prob

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{1103: 0.051695514,
 1104: 0.21479227,
 1112: 0.2970374,
 1116: 0.18241143,
 1120: 0.46755365,
 1124: 0.3781771,
 1129: 0.16730592,
 1136: 0.0637417,
 1151: 0.07693899,
 1159: 0.06391439,
 1161: 0.16767843,
 1163: 0.19682776999999996,
 1166: 0.07582379,
 1168: 0.055093855,
 1172: 0.1619786,
 1174: 0.05738225,
 1181: 0.2878635,
 1209: 0.06594069,
 1211: 0.44453818,
 1222: 0.25420314,
 1228: 0.23791534000000003,
 1231: 0.18331260000000005,
 1234: 0.18331260000000005,
 1235: 0.20628229,
 1240: 0.07029306999999996,
 1242: 0.46442088,
 1246: 0.40326649999999997,
 1255: 0.08982290000000004,
 1260: 0.17284290000000002,
 1261: 0.19170010000000004,
 1266: 0.18331260000000005,
 1272: 0.09919270000000002,
 1274: 0.19698210000000005,
 1276: 0.11842050000000004,
 1277: 0.18331260000000005,
 1286: 0.06950425999999998,
 1293: 0.15543883999999997,
 1308: 0.07943060000000002,
 1313: 0.06019629999999998,
 1314: 0.15645719999999996,
 1323: 0.07655080000000003,
 1326: 0.14790356000000004,
 1344: 0.1479035

In [352]:
max(minimum_prob, key=minimum_prob.get)

1120